# Classification on CIFAR and ImageNet

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!nvidia-smi

Thu Apr 14 00:41:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import sys

# check whether run in Colab
root = "."
if "google.colab" in sys.modules:
    print("Running in Colab.")
    !pip3 install matplotlib
    !pip3 install einops==0.3.0
    !pip3 install timm==0.4.9
    #!git clone https://github.com/xxxnell/how-do-vits-work.git
    root = "/content/drive/MyDrive/11785_project"
    sys.path.append(root)

Running in Colab.
     |████████████████████████████████| 346 kB 12.1 MB/s 


In [3]:
import os
import time
import yaml
import copy
from pathlib import Path
import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
import models #This takes long

In [4]:

import ops.trains as trains
import ops.tests as tests
import ops.datasets as datasets
import ops.schedulers as schedulers
torch.cuda.empty_cache()

In [19]:
config_path = "%s/configs/cifar10_vit.yaml" % root
#config_path = "%s/configs/jiachen_config/flowers102_simple.yaml" % root
#config_path = "%s/configs/jiachen_config/svhn_simple.yaml" % root
#config_path = "%s/configs/svhn_simple.yaml" % root
#config_path = "%s/configs/cifar10_simple.yaml" % root
#config_path = "%s/configs/jiachen_config/cifar100_vit.yaml" % root
#config_path = "%s/configs/cifar100_vit.yaml" % root
#config_path = "%s/configs/imagenet_vit.yaml" % root
#You own config path. for example, if Jiachen wants to run some models using his custommade model on CIFAR:

# config_path = "%s/configs/imagenet_vit.yaml" % root

with open(config_path) as f:
    args = yaml.load(f)
    print(args)

{'dataset': {'name': 'cifar10', 'root': '../data', 'mean': [0.4914, 0.4822, 0.4465], 'std': [0.2023, 0.1994, 0.201], 'padding': 4, 'color_jitter': 0.0, 'auto_augment': 'rand-m9-n2-mstd1.0', 're_prob': 0.0}, 'train': {'warmup_epochs': 5, 'epochs': 300, 'batch_size': 96, 'max_norm': 5, 'smoothing': 0.1, 'mixup': {'num_classes': 10, 'mixup_alpha': 1.0, 'cutmix_alpha': 0.8, 'prob': 1.0}}, 'val': {'batch_size': 256, 'n_ff': 1}, 'model': {'stem': False, 'block': {'image_size': 32, 'patch_size': 2, 'sd': 0.1}}, 'optim': {'name': 'AdamW', 'lr': 0.000125, 'weight_decay': 0.05, 'scheduler': {'name': 'CosineAnnealingLR', 'T_max': 300, 'eta_min': 0}}, 'env': {}}


In [ ]:
#dataset_train = torchvision.datasets.ImageNet(root='../data', split='train',download=True)

In [14]:
dataset_args = copy.deepcopy(args).get("dataset")
train_args = copy.deepcopy(args).get("train")
val_args = copy.deepcopy(args).get("val")
model_args = copy.deepcopy(args).get("model")
optim_args = copy.deepcopy(args).get("optim")
env_args = copy.deepcopy(args).get("env")

In [15]:
print(dataset_args)
dataset_train, dataset_test = datasets.get_dataset(**dataset_args, download=True)
dataset_name = dataset_args["name"]
num_classes = len(dataset_train.classes)
if (dataset_name == "svhn"):
  num_classes = 10

dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=train_args.get("num_workers", 4), 
                           batch_size=train_args.get("batch_size", 128))
dataset_test = DataLoader(dataset_test, 
                          num_workers=val_args.get("num_workers", 4), 
                          batch_size=val_args.get("batch_size", 128))

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    num_classes
))

{'name': 'cifar10', 'root': '../data', 'mean': [0.4914, 0.4822, 0.4465], 'std': [0.2023, 0.1994, 0.201], 'padding': 4}
You want:  cifar10


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Files already downloaded and verified
Files already downloaded and verified
Train: 50000, Test: 10000, Classes: 10


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [8]:
#print(type(dataset_test))
print(dataset_name)
print(type(dataset_train))



cifar10
<class 'torch.utils.data.dataloader.DataLoader'>


## Model

Use provided models:

In [16]:
# ResNet
name = "resnet_dnn_50"
# name = "resnet_dnn_101"

# ViT
#name = "vit_ti"
#name = "vit_s"

vit_kwargs = {  # for CIFAR
    "image_size": 32, 
    "patch_size": 16,
}
# For 11785: Add you new model in ./11785_project/models/new?model?name.py
# After adding new models, modify _init_.py so you model can be impored using models.get_model() method
model = models.get_model(name, num_classes=num_classes, 
                         stem=model_args.get("stem", False), **vit_kwargs)
# models.load(model, dataset_name, uid=current_time)

model: resnet_dnn_50 , params: 23.5M, output: [3, 10]


Or use `timm`:

In [ ]:
import timm
#Not using it for 11785
#model = timm.models.vision_transformer.VisionTransformer(
#    img_size=32, patch_size=2, num_classes=num_classes,  # for CIFAR
#    embed_dim=192, depth=12, num_heads=3, qkv_bias=False,  # ViT-Ti
#)
#model.name = "vit_ti"
#models.stats(model)

Parallelize the given `moodel` by splitting the input:

In [ ]:
#name = model.name
#model = nn.DataParallel(model)
#model.name = name

## Train

Define a TensorBoard writer:

In [17]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = os.path.join("runs", dataset_name, model.name, current_time)
writer = SummaryWriter(log_dir)

with open("%s/config.yaml" % log_dir, "w") as f:
    yaml.dump(args, f)
with open("%s/model.log" % log_dir, "w") as f:
    f.write(repr(model))

print("Create TensorBoard log dir: ", log_dir)

Create TensorBoard log dir:  runs/cifar10/resnet_dnn_50/20220425_225120


In [11]:
print(train_args)

{'warmup_epochs': 5, 'epochs': 300, 'batch_size': 96, 'max_norm': 5, 'smoothing': 0.1, 'mixup': {'num_classes': 10, 'mixup_alpha': 1.0, 'cutmix_alpha': 0.8, 'prob': 1.0}}


Train the model:

In [18]:
gpu = torch.cuda.is_available()
optimizer, train_scheduler = trains.get_optimizer(model, **optim_args)
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(dataset_train) * train_args.get("warmup_epochs", 0))

trains.train(model, optimizer,
             dataset_train, dataset_test,
             train_scheduler, warmup_scheduler,
             train_args, val_args, gpu,
             
             writer, 
             snapshot=-1, dataset_name=dataset_name, uid=current_time)  # Set `snapshot=N` to save snapshots every N epochs.

Train:   0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Train:  78%|███████▊  | 306/391 [04:54<01:21,  1.04it/s, acc=29.7564%, loss=1.9680, lr=0.0780]

KeyboardInterrupt: ignored

Save the model:

In [ ]:
models.save(model, dataset_name, current_time, optimizer=optimizer)

In [ ]:
print(*optim_args)

## Test

In [ ]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_test, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s.csv" % (dataset_name, model.name, current_time))
tests.save_metrics(metrics_dir, metrics_list)